Установка/настройка Kafka:
1)  Загрузка предустановленных вспомогательных скриптов с официального сайта для вспомогательных операций с топиками и проверки работоспособности
2)  Настройка окружения:

In [ ]:
    export KAFKA_ROOT_DIR=`pwd`
    cd ${KAFKA_ROOT_DIR}
    export KAFKA_DIR=${KAFKA_ROOT_DIR}/kafka_2.13-3.7.0
    export PATH=${KAFKA_DIR}/bin:${PATH}

Разворачиваем контейнер с Kafka - zookeeper сервер + 3 брокера (конфиг прилагается):

In [ ]:
#запуск контейнера с кластером
docker-compose -f docker-compose-cluster.yml up -d

#Проверяем, что все процессы запущены и работают
docker-compose -f docker-compose-cluster.yml ps

Создание топиков Kafka для дальнейшей работы. Так как у нас три брокера, то имеет смысл делать репликацию:

In [ ]:
#Создание топиков
    kafka-topics.sh --bootstrap-server localhost:9092 --create --topic raw_data --replication-factor 3 --partitions 3
    kafka-topics.sh --bootstrap-server localhost:9092 --create --topic processed_data --replication-factor 3 --partitions 3
    kafka-topics.sh --bootstrap-server localhost:9092 --create --topic visual_data --replication-factor 3 --partitions 3
    kafka-topics.sh --bootstrap-server localhost:9092 --create --topic ml_results --replication-factor 3 --partitions 3

#Проверка информации о топиках
    kafka-topics.sh --bootstrap-server localhost:9092 --describe --topic raw_data

#Проверяем, что все успешно - все 4 топика должны отображаться в результате этой команды:
    kafka-topics.sh --bootstrap-server localhost:9092 --list

Запуск основного приложения:
   1) Читает датасет из Spotify_Tracks_Dataset.csv входняка и записывает данные в 'raw_data' топик Кафки с помощью продьюсера.
   2) Processing Consumer - выделяет из всего датасета только треки в жанрах 'pop', 'acoustic', 'disco' and 'k-pop' и записывает их в топик 'processed_data'
   3) Visualize Consumer:
       a) Топ музыкальных жанров на основе суммарной популярности треков
       b) Топ-25 песен на основе популярности трека
       c) Выбираем Топ-3 жанра и для каждого показываем Топ-5 треков
       d) Показываем гистограмму распределения дополнительных характеристик треков - популярность, громкость, танцевальность и тд.
       e) Записываем финальный датафрейм, который использовался для визуализации в топик 'visual_data'
   4) ML Consumer - предсказание популярности трека на основе его дополнительных характеристик. Результат предсказаний записывается в топик 'ml_results'

In [ ]:
python3 Kafka_lab.py

Детали реализации:
1)  Отдельные классы для продьюсера и консьюмера.
    a) В продьюсере был испробован time.sleep() для эмуляции реального построчного поступления данных на вход.
    b) В процессе работы было использвоано несколько консьюмеров. Каждый со своим временным критерием остановки.
2)  Для одного из консьюмеров визуализация реализована с помощью plotly. Для МЛ консьюмера - matplotlib. 

![ОПИСАНИЕ](result_pics/top_songs_1.png)


![ОПИСАНИЕ](result_pics/top_songs_2.png)


![ОПИСАНИЕ](result_pics/pic1.png)


![ОПИСАНИЕ](result_pics/pic2.png)


![ОПИСАНИЕ](result_pics/pic4.png)


Задача Machine Learning на выбранном датасете



На основании характеристик треков будем предсказывать их популярность. Из фичей оставим только 'artists', 'track_genre', 'danceability', 'loudness', чтобы контролировать количество параметров при обучении, а в качестве таргета возьмем 'popularity'. Заэнкодим текстовые данные (их много, поэтому количество столбцов в датасете резко возрастет).

В условиях поставленной задачи сравним RandomForestRegressor и LinearRegressor по метрике MAE. 



RandomForestRegressor:

![ОПИСАНИЕ](result_pics/randomforest.png)

LinearRegressor:

![ОПИСАНИЕ](result_pics/linearregression.png)

Из визуализации видно, что random forest хорошо угадывает треки, имеющие среднюю популярность, но на очень популярных или очень непопулярных треках ошибается. 